In [ ]:
import warnings
import winsound 
duration = 500
frequency = 440
from itertools import combinations
import math
from math import sqrt
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import kstest,shapiro, ttest_ind, mannwhitneyu, chi2_contingency, fisher_exact
from statsmodels.formula.api import logit
import statsmodels.stats.proportion as sm
from statsmodels.tools import add_constant
import matplotlib
import pandas as pd
import feature_selector as selector
from feature_selector import FeatureSelector
# from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
font_set = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=12)
import waterfall
from matplotlib import pyplot
from matplotlib.ticker import MultipleLocator
# from ruamel.yaml import timestamp
from sklearn.feature_selection import SelectKBest, chi2, RFECV, SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_curve, auc, precision_recall_curve, precision_score, make_scorer, log_loss, roc_auc_score, mean_squared_error
from pandas import read_csv
from pandas import set_option
from sklearn import metrics, datasets, clone
import numpy as np
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, KBinsDiscretizer
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit, validation_curve, learning_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.calibration import calibration_curve
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.utils import shuffle
import seaborn as sns
from Delong_test import DelongTest
# from pyDeLong import DelongTest
from copy import copy
from shaphypetune import BoostSearch, BoostRFE, BoostRFA, BoostBoruta
from hyperopt import hp
from hyperopt import Trials
import joblib
from keras.models import load_model
from DCA import net_benefit_model, net_benefit_all, plot_DCA
from AUC_CI import auc_ci
from nri_f import nri

warnings.filterwarnings('ignore')

In [ ]:
# import original data
xlsx1_filepath = r'E:\Pystudy\Thyroid_P_N\data\results\data_use\mix\NHT_mix.xlsx'
xlsx2_filepath = r'E:\Pystudy\Thyroid_P_N\data\results\data_use\mix\PTMC_mix.xlsx'
df_1 = pd.read_excel(xlsx1_filepath, sheet_name=1)
df_2 = pd.read_excel(xlsx2_filepath, sheet_name=1)

winsound.Beep(frequency, duration)

In [ ]:
# concat data

rows_1, _ = df_1.shape
rows_2, _ = df_2.shape
df_1.insert(0, 'label', [0] * rows_1)
df_2.insert(0, 'label', [1] * rows_2)
df = pd.concat([df_1, df_2])
df = shuffle(df)
print("\n",'original concat：', df.shape)

tradition_labes = ['MAD_mm', 'Nodules',	'Shape', 'Margin',	'Cystic',	'ETE',	'Enhancement',	'T1_rCNR',	'T1C_rCNR', 'T1C_increase',	'T2_rCNR',	'B800_rCNR',	'B2000_rCNR',	'ADC_lesion',	'ADC_normal',	'ADC_rCNR',	'TI_RADS_score']
non_radiomics_labels = ['label', 'Name', 'MAD_mm', 'Nodules',	'Shape', 'Margin',	'Cystic',	'ETE',	'Enhancement',	'T1_rCNR',	'T1C_rCNR', 'T1C_increase',	'T2_rCNR',	'B800_rCNR',	'B2000_rCNR',	'ADC_lesion',	'ADC_normal',	'ADC_rCNR',	'TI_RADS_score']

y = df['label']
x = df.drop(non_radiomics_labels, axis = 1)
colNames = x.columns
x = x.astype(np.float64)
x = pd.DataFrame(x)
x.columns = colNames
print('original radiomics：', x.shape, "\n")

x_tradition = df[tradition_labes]
colName_tradition = x_tradition.columns
x_tradition = x_tradition.astype(np.float64)
x_tradition = pd.DataFrame(x_tradition)
x_tradition.columns = colName_tradition
print('original traditional data：', x_tradition.shape, "\n")



In [ ]:
print('original mixed data：', df.columns)
all_columns = df.columns

non_clean_names = ['label', 'Name', 'Nodules',	'Shape', 'Margin',	'Cystic',	'ETE',	'Enhancement', 'TI_RADS_score']
clean_names = all_columns.drop(non_clean_names)
print('needed clean columns：', clean_names)

In [ ]:
# def replace_outliers_with_median
def replace_outliers_with_median(df, column_name, class_column_name):
    outlier_counts = {label: 0 for label in df[class_column_name].unique()}
    for class_value in df[class_column_name].unique():
        class_df = df[df[class_column_name] == class_value]
        Q1 = class_df[column_name].quantile(0.25)
        Q3 = class_df[column_name].quantile(0.75)
        IQR = Q3 - Q1
        upper_bound = Q3 + 1.5 * IQR
        lower_bound = Q1 - 1.5 * IQR
        outlier_indices = class_df[(class_df[column_name] < lower_bound) | (class_df[column_name] > upper_bound)].index
        outlier_counts[class_value] += len(outlier_indices)
        df.loc[outlier_indices, column_name] = class_df[column_name].median()
    return df, outlier_counts


# replace_outliers_with_median according to label
outlier_counts_dict = {label: 0 for label in df['label'].unique()}
for col in clean_names:
    df, temp_outlier_counts = replace_outliers_with_median(df, col, 'label')    
    for label, count in temp_outlier_counts.items():
        outlier_counts_dict[label] += count

# diaplsy replace_outliers information
for label, total_outliers in outlier_counts_dict.items():
    total_data_for_label = len(df[df['label'] == label]) * len(clean_names)  
    percentage = (total_outliers / total_data_for_label) * 100
    print(f'label is{label} in data:')
    print(f'Total number of detected outliers: {total_outliers}')
    print(f'proportion of processed outliers in the labeled data: {percentage:.2f}%')
    print('----------------------------------')
        
# save
df.to_excel(r'E:\Pystudy\Thyroid_P_N\data\results\data_use\mix\outliers\non_split_chuyichangzhihou_all1.xlsx', index=False) 